# 🚗 Nhận Diện Biển Số Xe với YOLOv8
## Training trên Google Colab

Notebook này sẽ train model YOLOv8 để nhận diện biển số xe sử dụng dataset của bạn.

## 1. Cài đặt thư viện cần thiết

In [ ]:
# Cài đặt ultralytics và các thư viện cần thiết
!pip install ultralytics
!pip install roboflow

# Import thư viện
import torch
import os
import shutil
from ultralytics import YOLO
from IPython.display import Image, display
import matplotlib.pyplot as plt

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 2. Kết nối Google Drive (Tùy chọn)

In [ ]:
# Mount Google Drive để lưu trữ dataset và kết quả
from google.colab import drive
drive.mount('/content/drive')

# Tạo thư mục làm việc
work_dir = '/content/license_plate_detection'
os.makedirs(work_dir, exist_ok=True)
os.chdir(work_dir)
print(f"Working directory: {os.getcwd()}")

## 3. Upload Dataset
### Cách 1: Clone từ GitHub (Khuyến nghị)

In [ ]:
# Clone repository từ GitHub (thay YOUR_GITHUB_USERNAME và YOUR_REPO_NAME)
# !git clone https://github.com/YOUR_GITHUB_USERNAME/NhanDienBSX.git
# os.chdir('NhanDienBSX')

# Hoặc download từ link trực tiếp
print("Nếu bạn có GitHub repo, uncomment và sửa lại dòng git clone ở trên")
print("Hoặc sử dụng cách upload file ở cell tiếp theo")

### Cách 2: Upload thủ công

In [ ]:
# Upload files từ máy tính
from google.colab import files

print("Upload file mydata.yaml:")
uploaded = files.upload()

print("\nUpload dataset folder (nén thành .zip trước):")
uploaded_dataset = files.upload()

# Giải nén dataset nếu cần
import zipfile
for filename in uploaded_dataset.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print(f"Đã giải nén {filename}")

## 4. Tạo file cấu hình dataset

In [ ]:
# Tạo file mydata.yaml với đường dẫn tuyệt đối
import os
current_dir = os.getcwd()
dataset_path = os.path.join(current_dir, 'dataset')

yaml_content = f"""
# Dataset configuration for License Plate Detection
path: {dataset_path}  # dataset root dir (absolute path)
train: images/train  # train images (relative to 'path')
val: images/val  # val images (relative to 'path')
test:  # test images (optional)

# Classes
names:
  0: Bien_So

# Number of classes
nc: 1
"""

with open('mydata.yaml', 'w', encoding='utf-8') as f:
    f.write(yaml_content)

print(f"Đã tạo file mydata.yaml với đường dẫn: {dataset_path}")
print("Nội dung file:")
with open('mydata.yaml', 'r', encoding='utf-8') as f:
    print(f.read())

## 5. Kiểm tra cấu trúc dataset

In [ ]:
# Kiểm tra cấu trúc thư mục
def check_dataset_structure():
    required_dirs = [
        'dataset/images/train',
        'dataset/images/val',
        'dataset/labels/train',
        'dataset/labels/val'
    ]
    
    print("Kiểm tra cấu trúc dataset:")
    all_good = True
    for dir_path in required_dirs:
        if os.path.exists(dir_path):
            files = [f for f in os.listdir(dir_path) if f.endswith(('.jpg', '.png', '.jpeg', '.txt'))]
            file_count = len(files)
            print(f"✅ {dir_path}: {file_count} files")
            if file_count == 0:
                print(f"   ⚠️  Thư mục trống!")
                all_good = False
        else:
            print(f"❌ {dir_path}: Không tồn tại")
            all_good = False
    
    # Kiểm tra tương ứng giữa images và labels
    train_images = [f.split('.')[0] for f in os.listdir('dataset/images/train') if f.endswith(('.jpg', '.png', '.jpeg'))]
    train_labels = [f.split('.')[0] for f in os.listdir('dataset/labels/train') if f.endswith('.txt')]
    val_images = [f.split('.')[0] for f in os.listdir('dataset/images/val') if f.endswith(('.jpg', '.png', '.jpeg'))]
    val_labels = [f.split('.')[0] for f in os.listdir('dataset/labels/val') if f.endswith('.txt')]
    
    print(f"\n📊 Thống kê dataset:")
    print(f"   Train: {len(train_images)} images, {len(train_labels)} labels")
    print(f"   Val: {len(val_images)} images, {len(val_labels)} labels")
    
    if len(train_images) != len(train_labels):
        print(f"   ⚠️  Train: Số lượng images và labels không khớp!")
        all_good = False
    if len(val_images) != len(val_labels):
        print(f"   ⚠️  Val: Số lượng images và labels không khớp!")
        all_good = False
    
    if all_good:
        print("\n🎉 Dataset đã sẵn sàng cho training!")
    else:
        print("\n❌ Dataset có vấn đề, cần kiểm tra lại!")
    
    return all_good

dataset_ready = check_dataset_structure()

# Hiển thị cấu trúc thư mục
print("\n📁 Cấu trúc thư mục:")
!find . -type d -name "*" | head -20

## 6. Training Model

In [ ]:
# Kiểm tra dataset trước khi training
if not dataset_ready:
    print("❌ Dataset chưa sẵn sàng! Vui lòng kiểm tra lại.")
    raise Exception("Dataset không hợp lệ")

# Kiểm tra file mydata.yaml
if not os.path.exists('mydata.yaml'):
    print("❌ File mydata.yaml không tồn tại!")
    raise Exception("File cấu hình dataset không tồn tại")

# Load pretrained YOLO model
print("📥 Downloading pretrained model...")
model = YOLO("yolov8x.pt")  # hoặc yolov8n.pt, yolov8s.pt, yolov8m.pt, yolov8l.pt

# Training parameters
EPOCHS = 50
IMG_SIZE = 640
BATCH_SIZE = 16  # Giảm nếu bị out of memory

print(f"\n🚀 Bắt đầu training với {EPOCHS} epochs...")
print(f"📱 Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")
print(f"🖼️  Image size: {IMG_SIZE}")
print(f"📦 Batch size: {BATCH_SIZE}")

# Validate dataset trước khi train
try:
    print("\n🔍 Validating dataset...")
    model.val(data="mydata.yaml", split='val')
    print("✅ Dataset validation thành công!")
except Exception as e:
    print(f"❌ Dataset validation failed: {e}")
    print("Kiểm tra lại đường dẫn và cấu trúc dataset")
    raise

# Train the model
print("\n🏋️ Bắt đầu training...")
results = model.train(
    data="mydata.yaml",
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=0 if torch.cuda.is_available() else 'cpu',
    project='/content/runs',
    name='license_plate_detection',
    save=True,
    save_period=10,  # Lưu checkpoint mỗi 10 epochs
    patience=20,  # Early stopping
    verbose=True,
    plots=True  # Tạo plots
)

print("\n🎉 Training hoàn thành!")

## 7. Xem kết quả training

In [ ]:
# Hiển thị kết quả training
results_dir = '/content/runs/license_plate_detection'

# Tìm thư mục kết quả mới nhất
import glob
result_folders = glob.glob('/content/runs/license_plate_detection*')
if result_folders:
    latest_folder = max(result_folders, key=os.path.getctime)
    print(f"Thư mục kết quả: {latest_folder}")
    
    # Hiển thị các biểu đồ training
    plots = ['results.png', 'confusion_matrix.png', 'train_batch0.png', 'val_batch0_pred.png']
    
    for plot in plots:
        plot_path = os.path.join(latest_folder, plot)
        if os.path.exists(plot_path):
            print(f"\n📊 {plot}:")
            display(Image(plot_path))
        else:
            print(f"❌ Không tìm thấy {plot}")
else:
    print("❌ Không tìm thấy thư mục kết quả")

## 8. Test model với ảnh mẫu

In [ ]:
# Load model đã train
best_model_path = os.path.join(latest_folder, 'weights', 'best.pt')
if os.path.exists(best_model_path):
    trained_model = YOLO(best_model_path)
    print(f"✅ Đã load model: {best_model_path}")
    
    # Test với ảnh validation
    val_images = glob.glob('dataset/images/val/*.jpg') + glob.glob('dataset/images/val/*.png')
    
    if val_images:
        test_image = val_images[0]
        print(f"\n🔍 Test với ảnh: {test_image}")
        
        # Predict
        results = trained_model(test_image)
        
        # Hiển thị kết quả
        results[0].show()
        
        # Lưu kết quả
        results[0].save('prediction_result.jpg')
        display(Image('prediction_result.jpg'))
    else:
        print("❌ Không tìm thấy ảnh test")
else:
    print("❌ Không tìm thấy model đã train")

## 9. Download model và kết quả

In [ ]:
# Nén và download kết quả
import shutil

if result_folders:
    # Tạo file zip chứa kết quả
    shutil.make_archive('license_plate_model', 'zip', latest_folder)
    
    # Download file zip
    files.download('license_plate_model.zip')
    
    # Download riêng file model tốt nhất
    if os.path.exists(best_model_path):
        shutil.copy(best_model_path, 'best_license_plate_model.pt')
        files.download('best_license_plate_model.pt')
    
    print("✅ Đã download model và kết quả training!")
    print(f"📁 Model path: {best_model_path}")
    print(f"📊 Results folder: {latest_folder}")
else:
    print("❌ Không có kết quả để download")

## 10. Lưu vào Google Drive

In [ ]:
# Lưu kết quả vào Google Drive
drive_save_path = '/content/drive/MyDrive/license_plate_detection_results'
os.makedirs(drive_save_path, exist_ok=True)

if result_folders:
    # Copy toàn bộ kết quả
    shutil.copytree(latest_folder, os.path.join(drive_save_path, 'training_results'), dirs_exist_ok=True)
    
    # Copy model tốt nhất
    if os.path.exists(best_model_path):
        shutil.copy(best_model_path, os.path.join(drive_save_path, 'best_model.pt'))
    
    print(f"✅ Đã lưu kết quả vào Google Drive: {drive_save_path}")
else:
    print("❌ Không có kết quả để lưu")

print("\n🎉 Hoàn thành! Model đã được train và lưu trữ.")